1.	Python 
1)	Написать на python генератор первых n чисел фибоначи.


In [13]:
def fibonacci_generator(n):
    fibonacci = [0, 1]
    for i in range(0,n):
        fibonacci.append(fibonacci[i] + fibonacci[i+1])
    return fibonacci

2.	Написать Sql-запросы
1)	Вывести отдел с наибольшим числом сотрудников;
2)	Вывести список сотрудников, получающих заработную плату выше, чем у руководителя;


In [ ]:
%%sql sqlite://
SELECT 
    d.name AS department_name,
    COUNT(p.id) AS employee_count
FROM 
    Department d
JOIN 
    Personal p ON d.id = p.id_dep
GROUP BY 
    d.id, d.name
ORDER BY 
    employee_count DESC
LIMIT 1;

In [ ]:
%%sql sqlite://
SELECT 
    emp.id AS employee_id,
    emp.name AS employee_name,
    emp.sal AS employee_salary,
    mgr.name AS manager_name,
    mgr.sal AS manager_salary
FROM 
    Personal emp
JOIN 
    Personal mgr ON emp.id_head = mgr.id
WHERE 
    emp.sal > mgr.sal;

3.	Задачка на определение отрасли компании

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from lightgbm import LGBMClassifier

#Загрузим данные
pays = pd.read_csv('pays.csv')
inn_info = pd.read_csv('inn_info_public.csv')

#Объединеним данные
pays_agg = pays.groupby('hash_inn_kt').agg({
    'count': ['mean', 'sum'], 
    'sum': ['mean', 'sum'],
    'week': 'nunique'
}).reset_index()
pays_agg.columns = ['hash_inn'] + [f'feature_{i}' for i in range(len(pays_agg.columns)-1)]
data = pd.merge(inn_info, pays_agg, how='left', on='hash_inn')

#Обработаем целевую переменную
data_train = data[data['okved2'] != -1]
data_test = data[data['okved2'] == -1]

#Разделим данные на X и y
X = data_train.drop(columns=['okved2', 'hash_inn'])
y = data_train['okved2']
X_test = data_test.drop(columns=['okved2', 'hash_inn'])

#Обученим модели
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
model = LGBMClassifier()
model.fit(X_train, y_train)

#Оценим модель
y_pred = model.predict(X_val)

#Предсказание модели
data_test['y'] = model.predict(X_test)

#Отсортируем данные по столбцу 'hash_inn' в порядке возрастания
sorted_data_test = data_test.sort_values(by='hash_inn', ascending=True)

#Сохраненим полученный результат в CSV-файл
sorted_data_test[['hash_inn', 'y']].to_csv('BogdanNeverov-18112024-SberIndustry.csv', index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000585 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1135
[LightGBM] [Info] Number of data points in the train set: 121061, number of used features: 6
[LightGBM] [Info] Start training from score -5.288953
[LightGBM] [Info] Start training from score -8.066464
[LightGBM] [Info] Start training from score -7.484542
[LightGBM] [Info] Start training from score -5.256744
[LightGBM] [Info] Start training from score -3.053550
[LightGBM] [Info] Start training from score -6.467608
[LightGBM] [Info] Start training from score -4.185443
[LightGBM] [Info] Start training from score -5.515786
[LightGBM] [Info] Start training from score -3.545534
[LightGBM] [Info] Start training from score -4.449165
[LightGBM] [Info] Start training from score -5.078657
[LightGBM] [Info] Start training from score -4.036892

/var/folders/lf/x86pzptd3pv7z45l56wggyjc0000gn/T/ipykernel_6494/3156699921.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['y'] = model.predict(X_test)
